In [206]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns


In [215]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

In [218]:
def fill_nan(x):    
    x['Age'] = x['Age'].fillna(x['Age'].mean())
    x['Cabin'] = x['Cabin'].isna().astype(int)
    x['Fare'] =x['Fare'].fillna(x['Fare'].mean())
    x['Embarked'] = x['Embarked'].replace({'S':1,'C':2,'Q':3}).fillna(1)
    return x 

In [272]:
def get_title(x): 
    x['title'] = x['Name'].str.split(',',expand = True)[1].str.split('.',expand = True)[0]
    values_s = x['title'].unique()
    mr_values = ['Don','Rev','Dr','Major','Sir','Col','Capt','Jonkheer']
    title_list = []
    for title in x['title']:
        if title in mr_values:
            title_list.append('mr')
        else:
            print('whyyyyyyy')
        #if title == 'Mr':
            #print(type(title))
    return title_list
    ##dataset['title'] = dataset['title'].replace(['Mme','Lady'],'Mrs')
    #dataset['title'] = dataset['title'].replace(['Ms','Mlle'],'Miss')



In [273]:
get_title(train_data)

whyyyyyyy
whyyyyyyy
whyyyyyyy
whyyyyyyy
whyyyyyyy
whyyyyyyy
whyyyyyyy
whyyyyyyy
whyyyyyyy
whyyyyyyy
whyyyyyyy
whyyyyyyy
whyyyyyyy
whyyyyyyy
whyyyyyyy
whyyyyyyy
whyyyyyyy
whyyyyyyy
whyyyyyyy
whyyyyyyy
whyyyyyyy
whyyyyyyy
whyyyyyyy
whyyyyyyy
whyyyyyyy
whyyyyyyy
whyyyyyyy
whyyyyyyy
whyyyyyyy
whyyyyyyy
whyyyyyyy
whyyyyyyy
whyyyyyyy
whyyyyyyy
whyyyyyyy
whyyyyyyy
whyyyyyyy
whyyyyyyy
whyyyyyyy
whyyyyyyy
whyyyyyyy
whyyyyyyy
whyyyyyyy
whyyyyyyy
whyyyyyyy
whyyyyyyy
whyyyyyyy
whyyyyyyy
whyyyyyyy
whyyyyyyy
whyyyyyyy
whyyyyyyy
whyyyyyyy
whyyyyyyy
whyyyyyyy
whyyyyyyy
whyyyyyyy
whyyyyyyy
whyyyyyyy
whyyyyyyy
whyyyyyyy
whyyyyyyy
whyyyyyyy
whyyyyyyy
whyyyyyyy
whyyyyyyy
whyyyyyyy
whyyyyyyy
whyyyyyyy
whyyyyyyy
whyyyyyyy
whyyyyyyy
whyyyyyyy
whyyyyyyy
whyyyyyyy
whyyyyyyy
whyyyyyyy
whyyyyyyy
whyyyyyyy
whyyyyyyy
whyyyyyyy
whyyyyyyy
whyyyyyyy
whyyyyyyy
whyyyyyyy
whyyyyyyy
whyyyyyyy
whyyyyyyy
whyyyyyyy
whyyyyyyy
whyyyyyyy
whyyyyyyy
whyyyyyyy
whyyyyyyy
whyyyyyyy
whyyyyyyy
whyyyyyyy
whyyyyyyy
whyyyyyyy
whyyyyyyy


[]